In [1]:
import pandas as pd

In [ ]:
def get_direct_link(shared_link):
    return 'https://drive.google.com/uc?export=download&id=' + shared_link.split('/')[-2]

link = "https://drive.google.com/file/d/125peVnvL3G7_hv_RGNIT0J416sGm6kMo/view?usp=drive_link"

direct_link = get_direct_link(link)
df = pd.read_csv(direct_link)

In [3]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [4]:
teams = pd.DataFrame(pd.concat([df['HomeTeam'],df['AwayTeam']]).unique())
teams.columns = ['original_names']
teams['search_query'] = teams['original_names'] + " football club wikipedia"

In [5]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

def get_first_search_result_title(query):
    query = urllib.parse.quote_plus(query)
    url = f"https://www.google.com/search?q={query}"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        'Accept-Language': 'en-US,en;q=0.5'
    }
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    first_result = soup.find("div", {"class": "yuRUbf"})
    if first_result:
        title = first_result.find("h3").text
        return title.strip()

    return None

count = 0
for index, row in teams.iterrows():
    if index < count:
        continue
    query = row["search_query"]
    result_title = get_first_search_result_title(query)
    if not result_title:
        print(count,index,row['original_names'])
        break
    teams.at[index, "wiki_title"] = result_title if result_title else "No search results found"

In [6]:
teams["wiki_title"] = teams["wiki_title"].str.replace(' - Wikipedia', '')

In [7]:
teams.to_csv('teams.csv', index = False)